In [7]:
import pandas as pd

# 读取原始文档，生成分段数据
- 原始文档为`fin_articles.csv`,包含列：src_web,keywords,text,len_text
- 按照换行符分割文档
- 分段数据包含：src_idx,keeywords,text,mentions

In [ ]:
# 准备语料
df=pd.read_csv(r'fin_articles.csv')
df.head(2)

In [ ]:
# 文本按换行符分割
all_src_idx=[]
all_kws=[]
all_texts=[]
for index,row in df.iterrows():
    texts=row['text'].split('\n') if isinstance(row['text'],str) else []
    # texts=[t for t in texts if len(t)>5] #过短的直接剔除
    all_src_idx.extend([index] * len(texts)) # 所属原文idx
    all_kws.extend([row['keywords']] * len(texts))
    all_texts.extend(texts)
print(len(all_texts))
# segment_articles
df_seg=pd.DataFrame({'src_idx':all_src_idx, 'keywords':all_kws, 'text':all_texts})
df_seg.head(20)


In [ ]:
# df_seg[df_seg['text'].str.len()<5].head()
df_seg=df_seg[df_seg['text'].str.len()>5] #过短的句子没有意义 空字符串会报错
df_seg.tail(10)

In [ ]:
# 删除一部分数据后，索引列变得不连续，重新索引
df_seg=df_seg.reset_index(drop=True)
df_seg.tail(10)

In [ ]:
df_seg.to_csv('segment_articles.csv',sep='\t')

# 读取存储的分段数据并NER
- 注意指定index_col=0，文件里面包含了index
- 总数据量10w左右，后期需要进行一定筛选，此处先全部识别出来
- `data\segment_articles_0127_with_entity.csv`为NER之后的文档

In [8]:
df_seg2=pd.read_csv(r'..\data\segment_articles_0127_with_entity.csv',sep='\t',index_col=0)
df_seg2.head()

,src_idx,keywords,text,mentions
0,0,信息泄露,消费火正旺 金融添把柴,O
1,0,信息泄露,年轻人消费方式的改变和信用消费意识的增强，推动消费金融行业井喷式增长,O
2,0,信息泄露,“现在工作这么累，想在休息日好好犒劳自己。只要有假期，我就会去世界各地旅行，已经去过韩国、日...,上海某律所/ORG;吴辰伟/PERSON;塞班岛/GPE;土耳其/GPE;以色列/GPE;日...
3,0,信息泄露,80后、90后等年轻一代消费方式的改变和信用消费意识的增强，是近些年我国消费金融行业蓬勃发展...,刘明露/PERSON;京东/GPE;淘宝/ORG;天津/GPE;沪江网校/ORG;山东临沂/GPE
4,0,信息泄露,银保监会近日下发通知，要求积极发展消费金融，适应多样化、多层次消费需求，提供和改进差异化金融...,中国人民大学重阳金融研究院/ORG;银保监会/ORG;董希淼/PERSON


In [11]:
print(df_seg2.shape) # 总量100181
print(df_seg2[df_seg2['mentions'].isnull()].shape) # 2000条未识别
print(df_seg2[df_seg2['mentions']=='O'].shape) #30029条无实体
print(df_seg2[df_seg2['mentions'].notnull()][df_seg2['mentions']!='O'].shape) #68152满足条件的

(100181, 4)
(2000, 4)
(30029, 4)
(68152, 4)


C:\Users\56386\AppData\Local\Temp\ipykernel_14592\2614035667.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(df_seg2[df_seg2['mentions'].notnull()][df_seg2['mentions']!='O'].shape)


In [9]:
df_entity=df_seg2[df_seg2['mentions'].notnull()][df_seg2['mentions']!='O']
df_entity.head(20)

C:\Users\56386\AppData\Local\Temp\ipykernel_1644\2520315626.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_entity=df_seg2[df_seg2['mentions'].notnull()][df_seg2['mentions']!='O']


,src_idx,keywords,text,mentions
2,0,信息泄露,“现在工作这么累，想在休息日好好犒劳自己。只要有假期，我就会去世界各地旅行，已经去过韩国、日...,上海某律所/ORG;吴辰伟/PERSON;塞班岛/GPE;土耳其/GPE;以色列/GPE;日...
3,0,信息泄露,80后、90后等年轻一代消费方式的改变和信用消费意识的增强，是近些年我国消费金融行业蓬勃发展...,刘明露/PERSON;京东/GPE;淘宝/ORG;天津/GPE;沪江网校/ORG;山东临沂/GPE
4,0,信息泄露,银保监会近日下发通知，要求积极发展消费金融，适应多样化、多层次消费需求，提供和改进差异化金融...,中国人民大学重阳金融研究院/ORG;银保监会/ORG;董希淼/PERSON
5,0,信息泄露,苏宁金融研究院互联网金融研究中心主任薛洪言认为，正是近期的消费升级大大促进了消费金融的发展；...,薛洪言/PERSON;苏宁金融研究院互联网金融研究中心/ORG
8,0,信息泄露,北京某事业单位职工王晶去年从银行借了一笔消费贷款，额度100万元，贷款直接到账方便支取。今年...,王晶/PERSON;王晶想/PERSON;北京/GPE
9,0,信息泄露,“当前居民住房杠杆率较高，需要密切关注过度借贷导致的居民杠杆率继续上升的问题。此前还有部分消...,何飞/PERSON;交通银行金融研究中心/ORG
12,0,信息泄露,在江苏一家医院工作的赵景明陆续收到某家分期平台的催款短信，但赵景明从未注册过且未在该平台购买...,江苏/GPE;赵景明/PERSON
14,0,信息泄露,北京消费者胡杨今年5月准备购买一部价值3100多元的一款手机，在朝阳大悦城附近一个手机店里，...,胡杨/PERSON;北京/GPE
16,0,信息泄露,薛洪言说，受消费观念、收入增长预期、信用环境成熟度甚至城镇化水平、人口结构等多方面因素影响，...,薛洪/PERSON
17,0,信息泄露,针对过度借贷可能导致的居民杠杆率较快上升的问题，何飞说，既要密切关注互联网消费金融的发展，取...,何飞/PERSON


In [14]:
df_entity.tail(20)

,src_idx,keywords,text,mentions
100119,4499,非法集资,可以说，当前中国金融体系出现的一系列问题，包括资产负债表大跃进、影子银行病毒式生长、金融腐败...,中国/GPE
100121,4499,非法集资,从全局鸟瞰中国的国家资产负债表，金融部门主要由商业银行构成，其资产配置组合“信贷+非标+债券...,中国/GPE
100124,4499,非法集资,然而，中国金融体系的会计难题，则大大阻碍了债务出清机制的发挥。虽然发达国家也存在不少的国有金...,中国/GPE
100125,4499,非法集资,可以说，货币大国仅仅在不到五年内就迅速崛起（M2相当于美日总和，占全球近乎三分之一），根本就...,美日/GPE
100127,4499,非法集资,更为尴尬的是，当前中国金融企业的规模扩张冲动和金融乱相，只能依靠金融反腐（比如AB、HNA、...,中国/GPE
100128,4499,非法集资,所以，利率市场化“穿透”到底，是利率双轨制的并轨，同时需要产权二元的并轨。否则，仅仅是利率价...,中国/GPE
100132,4499,非法集资,然而利率市场化已经接近尾声。新上任的央行行长，正努力带领银行家们走出乱象丛生的影子银行“中世...,央行/ORG
100133,4499,非法集资,看似一个简单的利率放开，却是中国金融体系一次颠覆式的“思想启蒙”或“金融革命”。从学理上看，...,中国/GPE
100145,4499,非法集资,市场化利率时代是市场环境多变的时代，波特的线性静态竞争战略将遭遇较大的挑战（几年前波特的战略...,波特/ORG
100150,4499,非法集资,于是，装修豪华的银行网点，越来越像二战时期法国斥巨资修建的马奇诺防线，所起的可能就是个心理上...,法国/GPE


In [13]:
import pickle
m2e=dict()
with open('D:\desktop\数据集收集\m2e\m2e.txt','r',encoding='utf-8') as f:
    lines=f.readlines()
    for l in lines:
        mention,entity=l.strip().split('\t')
        if mention in m2e:
            m2e[mention].add(entity)
        else:
            m2e[mention]=set([entity])
for k in m2e:
    value=m2e[k]
    m2e[k]={i:entity for i,entity in enumerate(value)}
print('size of m2e:',len(m2e))
with open('../data/m2e.pkl','wb') as fr: # 字典key为mention，value为候选实体列表的字典
    pickle.dump(m2e, fr)

size of m2e: 669496


In [14]:
with open('../data/m2e.pkl','rb') as fr2: # 字典key为mention，value为候选实体列表的字典
    m2e2=pickle.load(fr2)
anno_corpus=[]
for _,row in df_entity.iterrows():
    mentions=row['mentions'].split(';')
    for men_tuple in mentions:
        men=men_tuple.split('/')[0] # 实体mention
        # 1. 非单字；2.能搜到；3.有多种含义
        if len(men)>1 and men in m2e2 and len(m2e2[men])>1: # 存在歧义情况的非单字实体作为语料  其实这些没有歧义的是否可以考虑作为辅助语料
            anno_corpus.append([row['src_idx'],
                                row['keywords'],
                                row['text'], 
                                men_tuple,
                                m2e2[men]])
df_anno=pd.DataFrame(anno_corpus,columns=['src_idx','keywords','text','mention','entity_list'])
df_anno.head(20)

,src_idx,keywords,text,mention,entity_list
0,0,信息泄露,“现在工作这么累，想在休息日好好犒劳自己。只要有假期，我就会去世界各地旅行，已经去过韩国、日...,日本/GPE,"{0: '日本（中国旅游出版社图书）', 1: '日本（山名）', 2: '日本（李安强编著..."
1,0,信息泄露,“现在工作这么累，想在休息日好好犒劳自己。只要有假期，我就会去世界各地旅行，已经去过韩国、日...,韩国/GPE,"{0: '韩国（战国七雄之一）', 1: '韩国（中国明朝藩国）', 2: '韩国（周朝诸侯..."
2,0,信息泄露,北京某事业单位职工王晶去年从银行借了一笔消费贷款，额度100万元，贷款直接到账方便支取。今年...,王晶/PERSON,"{0: '王晶（首都医科大学宣武医院主任医师）', 1: '王晶（天狼伟业（北京）国际咨询管..."
3,0,信息泄露,北京某事业单位职工王晶去年从银行借了一笔消费贷款，额度100万元，贷款直接到账方便支取。今年...,北京/GPE,"{0: '北京（太原古代的名称）', 1: '北京（开封古代的称呼）', 2: '北京（林妙..."
4,0,信息泄露,“当前居民住房杠杆率较高，需要密切关注过度借贷导致的居民杠杆率继续上升的问题。此前还有部分消...,何飞/PERSON,"{0: '何飞（兴义民族师范学院教师）', 1: '何飞（作家）', 2: '何飞（华盛网络..."
5,0,信息泄露,北京消费者胡杨今年5月准备购买一部价值3100多元的一款手机，在朝阳大悦城附近一个手机店里，...,胡杨/PERSON,"{0: '胡杨（西南交通大学经济管理学院副教授）', 1: '胡杨（新疆歌手）', 2: '..."
6,0,信息泄露,北京消费者胡杨今年5月准备购买一部价值3100多元的一款手机，在朝阳大悦城附近一个手机店里，...,北京/GPE,"{0: '北京（太原古代的名称）', 1: '北京（开封古代的称呼）', 2: '北京（林妙..."
7,0,信息泄露,薛洪言说，受消费观念、收入增长预期、信用环境成熟度甚至城镇化水平、人口结构等多方面因素影响，...,薛洪/PERSON,"{0: '薛洪（薛洪）', 1: '薛洪（东汉末人物）'}"
8,0,信息泄露,针对过度借贷可能导致的居民杠杆率较快上升的问题，何飞说，既要密切关注互联网消费金融的发展，取...,何飞/PERSON,"{0: '何飞（兴义民族师范学院教师）', 1: '何飞（作家）', 2: '何飞（华盛网络..."
9,0,信息泄露,何飞认为，解决信贷流向不清的问题，在消费贷款种类上应鼓励发展商品贷，有效管控现金贷，加大对现...,何飞/PERSON,"{0: '何飞（兴义民族师范学院教师）', 1: '何飞（作家）', 2: '何飞（华盛网络..."


In [15]:
df_anno.shape # (30990, 5)

(30990, 5)

In [17]:
m2e2['潘建伟']

{0: '潘建伟（浙江师范大学特聘教授）', 1: '潘建伟（中国科学院院士、中国科学技术大学教授）'}

In [18]:
df_anno['annotation']=None
df_anno.head()

,src_idx,keywords,text,mention,entity_list,annotation
0,0,信息泄露,“现在工作这么累，想在休息日好好犒劳自己。只要有假期，我就会去世界各地旅行，已经去过韩国、日...,日本/GPE,"{0: '日本（中国旅游出版社图书）', 1: '日本（山名）', 2: '日本（李安强编著...",None
1,0,信息泄露,“现在工作这么累，想在休息日好好犒劳自己。只要有假期，我就会去世界各地旅行，已经去过韩国、日...,韩国/GPE,"{0: '韩国（战国七雄之一）', 1: '韩国（中国明朝藩国）', 2: '韩国（周朝诸侯...",None
2,0,信息泄露,北京某事业单位职工王晶去年从银行借了一笔消费贷款，额度100万元，贷款直接到账方便支取。今年...,王晶/PERSON,"{0: '王晶（首都医科大学宣武医院主任医师）', 1: '王晶（天狼伟业（北京）国际咨询管...",None
3,0,信息泄露,北京某事业单位职工王晶去年从银行借了一笔消费贷款，额度100万元，贷款直接到账方便支取。今年...,北京/GPE,"{0: '北京（太原古代的名称）', 1: '北京（开封古代的称呼）', 2: '北京（林妙...",None
4,0,信息泄露,“当前居民住房杠杆率较高，需要密切关注过度借贷导致的居民杠杆率继续上升的问题。此前还有部分消...,何飞/PERSON,"{0: '何飞（兴义民族师范学院教师）', 1: '何飞（作家）', 2: '何飞（华盛网络...",None


In [20]:
anno_dir='../data/seg_anno.csv'
df_anno.to_csv(anno_dir,sep='\t') # 默认会存储索引

In [21]:
df_r=pd.read_csv(anno_dir,sep='\t',index_col=0)
df_r.tail()

,src_idx,keywords,text,mention,entity_list,annotation
30985,4499,非法集资,看似一个简单的利率放开，却是中国金融体系一次颠覆式的“思想启蒙”或“金融革命”。从学理上看，...,中国/GPE,"{0: '中国（卓依婷演唱歌曲）', 1: '中国（郭光主编图书）', 2: '中国（197...",NaN
30986,4499,非法集资,市场化利率时代是市场环境多变的时代，波特的线性静态竞争战略将遭遇较大的挑战（几年前波特的战略...,波特/ORG,"{0: '波特（服装品牌）', 1: '波特（美国NBA运动员）', 2: '波特（设备每秒...",NaN
30987,4499,非法集资,于是，装修豪华的银行网点，越来越像二战时期法国斥巨资修建的马奇诺防线，所起的可能就是个心理上...,法国/GPE,"{0: '法国（欧洲西部国家）', 1: '法国（APA Publications主编图书）'}",NaN
30988,4500,非法集资,财新网04月24日电: 沪指开盘报3063.44点，跌幅 0.26%。,财新网/ORG,"{0: '财新网（网站）', 1: '财新网（手机软件）'}",NaN
30989,4500,非法集资,消息面上，央行称虚拟货币交易场所已基本实现无风险退出；海南鼓励房企将住宅土储用于发展其他产业...,海南/GPE,"{0: '海南（大韩民国海南郡）', 1: '海南（黄学坚2004年出版图书）', 2: '...",NaN
